# Prelude

In [ ]:
# Enable interactive plots (%matplotlib -l to list backends)
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

from bgcellmodels.common import analysis, signal, units, morphology, treeutils
from bgcellmodels.mechanisms import synapses

import neuron; h = neuron.h
import bluepyopt.ephys as ephys

import pprint # or turn on for non-print statements using %pprint
pp = pprint.PrettyPrinter(indent=2)

# print code version (hash of checked out version)
print("\nCurrent commit:")
!git log -1
print("\nChanges since last commit:")
!git status --short

# print date and time of script execution
import datetime
print("\nNotebook executed at at {} in following directory:".format(datetime.datetime.now()))
%cd ..

# Create cell

In [ ]:
# Use GPe cell class used by PyNN
# import bgcellmodels.models.GPe.Fujita2011.fujita_pynn_model as fujita
# cell = fujita.GpeProtoType.model(**fujita.GpeProtoType.default_parameters)
# icell = cell.icell

import bgcellmodels.models.GPe.Fujita2011.fujita_model as fujita
icell = h.FujitaGPE() # new instance of cell template
icell.setparams_corbit_2016()

somatic = list(icell.somatic)
dendritic = list(icell.basal)
axonal = list(icell.axonal)

soma = somatic[0]
# axon = axonal[0]
# dend = dendritic[0]

# Plot Morphology & Properties

Inspect cell structure to make sure everything is sane.

## Conductances

Plot using NEURON ModelView : `from neuron import gui` -> Tools -> ModelView.

In [ ]:
pp.pprint(fujita.mechs_params_dict)

In [ ]:
# Adjust conductances to test effect
n_adj = 1
for mech in ['CaHVA']: # HCN
    for sec in icell.all:
        if h.ismembrane(mech, sec=sec):
            n_adj += 1
            for seg in sec:
                # seg.gmax_HCN = 0.5 * seg.gmax_HCN
                print("{} : gmax_{} = {}".format(seg, mech, getattr(seg, 'gmax_'+mech)))

    print("Adjusted gmax_HCN in {} sections.".format(n_adj))

# Record & Plot Protocols

List of possible traces for recordings:

In [ ]:
trace_specs = {
    # Membrane voltages
    'V_soma': {'var':'v', 'sec':'soma', 'loc':0.5},
    # Membrane currents
#     'iCaH': {'var':'iCaH', 'mech':'CaHVA', 'sec':'dend', 'loc':0.5},
#     'iHCN': {'var':'iHCN', 'mech':'HCN', 'sec':'dend', 'loc':0.5},
#     'iHCN2': {'var':'iHCN', 'mech':'HCN2', 'sec':'dend', 'loc':0.5},
#     'iKCNQ': {'var':'iKCNQ', 'mech':'KCNQ', 'sec':'dend', 'loc':0.5},
#     'iKv2': {'var':'iKv2', 'mech':'Kv2', 'sec':'dend', 'loc':0.5},
#     'iKv3': {'var':'iKv3', 'mech':'Kv3', 'sec':'dend', 'loc':0.5},
#     'iKv4f': {'var':'iKv4f', 'mech':'Kv4f', 'sec':'dend', 'loc':0.5},
#     'iKv4s': {'var':'iKv4s', 'mech':'Kv4s', 'sec':'dend', 'loc':0.5},
#     'iNa': {'var':'iNa', 'mech':'NaF', 'sec':'dend', 'loc':0.5},
#     'iNa': {'var':'iNa', 'mech':'NaP', 'sec':'dend', 'loc':0.5},
#     'iSK': {'var':'iSK', 'mech':'SK', 'sec':'dend', 'loc':0.5},
    # Ion concentrations
#     'cai': {'var':'cai', 'sec':'dend', 'loc':0.5},
    # Gating variables
#     'mCaH': {'var':'m', 'mech':'CaHVA', 'sec':'dend', 'loc':0.5},
#     'taumCaH': {'var':'taum', 'mech':'CaHVA', 'sec':'dend', 'loc':0.5},
#     'minfCaH': {'var':'minf', 'mech':'CaHVA', 'sec':'dend', 'loc':0.5},
#     'mHCN': {'var':'m', 'mech':'HCN', 'sec':'dend', 'loc':0.5},
#     'mHCN2': {'var':'m', 'mech':'HCN2', 'sec':'dend', 'loc':0.5},
#     'mKCNQ': {'var':'m', 'mech':'KCNQ', 'sec':'dend', 'loc':0.5},
#     'mKv2': {'var':'m', 'mech':'Kv2', 'sec':'dend', 'loc':0.5},
#     'hKv2': {'var':'h', 'mech':'Kv2', 'sec':'dend', 'loc':0.5},
#     'mKv3': {'var':'m', 'mech':'Kv3', 'sec':'dend', 'loc':0.5},
#     'hKv3': {'var':'h', 'mech':'Kv3', 'sec':'dend', 'loc':0.5},
#     'mKv4f': {'var':'m', 'mech':'Kv4f', 'sec':'dend', 'loc':0.5},
#     'hKv4f': {'var':'h', 'mech':'Kv4f', 'sec':'dend', 'loc':0.5},
#     'mKv4s': {'var':'m', 'mech':'Kv4s', 'sec':'dend', 'loc':0.5},
#     'hKv4s': {'var':'h', 'mech':'Kv4s', 'sec':'dend', 'loc':0.5},
#     'mNaF': {'var':'m', 'mech':'NaF', 'sec':'dend', 'loc':0.5},
#     'hNaF': {'var':'h', 'mech':'NaF', 'sec':'dend', 'loc':0.5},
#     'sNaF': {'var':'s', 'mech':'NaF', 'sec':'dend', 'loc':0.5},
#     'mNaP': {'var':'m', 'mech':'NaP', 'sec':'dend', 'loc':0.5},
#     'hNaP': {'var':'h', 'mech':'NaP', 'sec':'dend', 'loc':0.5},
#     'sNaP': {'var':'s', 'mech':'NaP', 'sec':'dend', 'loc':0.5},
#     'mSK': {'var':'m', 'mech':'SK', 'sec':'dend', 'loc':0.5},
}

## Spontaneous Firing

Edgerton 2010, Fig. 2

Spontaneous firing for Arkypallidal cells in shown in:
- Abdi, Mallet et al (2015), Fig. 7 : f = 3 Hz
- Bogacz, Moraud, et al (2016), Fig. 3 : f = 2 Hz

In [ ]:
rec_secs = {
    'soma': soma,
}

trace_specs = {
    # Membrane voltages
    'V_soma': {'var':'v', 'sec':'soma', 'loc':0.5},
    # Ion concentrations
    'cai': {'var':'cai', 'sec':'soma', 'loc':0.5},
}

rec_dt = 0.05
vec_dict, markers = analysis.recordTraces(rec_secs, trace_specs, rec_dt)

# Custom recordings
vrec = h.Vector()
vrec.record(soma(0.5)._ref_v)
trec = h.Vector()
trec.record(h._ref_t)

In [ ]:
# Init and run simulation
h.dt = 0.025
h.celsius = 35.0
h.v_init = -68.0
h.tstop = 1000.0
h.init()
h.run()

In [ ]:
# Plot custom recordings
plt.figure()
plt.plot(trec.as_numpy(), vrec.as_numpy())
plt.xlim(0, h.tstop)
# plt.ylim(-80,60)
plt.xlabel('time (ms)')
plt.ylabel('$V_m$ (mV)')
plt.title('Spontaneous activity')

# Plot recorded traces
figs_vm = analysis.plotTraces(vec_dict, rec_dt, traceSharex=True) # yRange=(-80,40),

## Stimulation with 100 pA

See article Gunay (2008), Fig. 1 and Fig. 2.

In [ ]:
# Insert electrode
stim = h.IClamp(soma(0.5))

current = units.Quantity(100, 'pA')
scaled_current = units.to_nrn_units(current, h, 'amp', 'IClamp')
print("IClamp amplitude is: {}".format(scaled_current))

# Amplitude adjustment: soma surface was changed by factor 1 / 13.4^2 == pi*1^2 / pi*13.4^2
# However: remaining compartments were changed by smaller factor, so this is not good adjustment
surf_factor = 0.05

stim.delay = 1000
stim.dur = 1000
stim.amp = 0.1 * surf_factor # 100 pA = 0.1 nA

In [ ]:
# Init and run simulation
h.dt = 0.025
h.celsius = 35.0
h.v_init = -68.0
h.tstop = 3000.0
h.init()
h.run()

# Plot custom recordings
plt.figure()
plt.plot(trec.as_numpy(), vrec.as_numpy())
plt.xlim(0, h.tstop)
# plt.ylim(-80,60)
plt.xlabel('time (ms)')
plt.ylabel('$V_m$ (mV)')
plt.title('Spontaneous activity')

# Plot recorded traces
figs_vm = analysis.plotTraces(vec_dict, rec_dt, traceSharex=True) # yRange=(-80,40),

## Stimulation with -100 pA

See article Gunay (2008), Fig. 1 and Fig. 2.

In [ ]:
# Insert electrode
stim = h.IClamp(soma(0.5))

current = units.Quantity(100, 'pA')
scaled_current = units.to_nrn_units(current, h, 'amp', 'IClamp')
print("IClamp amplitude is: {}".format(scaled_current))

# Amplitude adjustment: soma surface was changed by factor 1 / 13.4^2 == pi*1^2 / pi*13.4^2
# However: remaining compartments were changed by smaller factor, so this is not good adjustment
surf_factor = 0.05

stim.delay = 1000
stim.dur = 1000
stim.amp = -scaled_current.magnitude * surf_factor # 100 pA = 0.1 nA

In [ ]:
# Init and run simulation
h.dt = 0.025
h.celsius = 35.0
h.v_init = -68.0
h.tstop = 3000.0
h.init()
h.run()

# Plot custom recordings
plt.figure()
plt.plot(trec.as_numpy(), vrec.as_numpy())
plt.xlim(0, h.tstop)
plt.ylim(-80,40)
plt.xlabel('time (ms)')
plt.ylabel('$V_m$ (mV)')
plt.title('Spontaneous activity')

# Plot recorded traces
figs_vm = analysis.plotTraces(vec_dict, rec_dt, traceSharex=True) # yRange=(-80,40),